In [ ]:
import pdfplumber
import pandas as pd

def extract_tables_from_pdf(pdf_path, output_excel):
    all_extracted_data = []
    pending_row = None  # Untuk menyimpan baris yang terpotong
    previous_headers = None  # Untuk menyimpan header jika kolom tabel terpotong

    def clean_text(text):
        if pd.isnull(text):
            return ''
        return str(text).strip().lower().replace('\n', ' ')

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()

            for table in tables:
                df = pd.DataFrame(table).applymap(clean_text)

                # Jika tabel memiliki header baru, simpan sebagai header
                if df.iloc[0].str.contains("pecahan", case=False, na=False).any():
                    previous_headers = df.iloc[0].tolist()
                    df = df[1:].reset_index(drop=True)

                # Jika halaman ini tidak memiliki header tetapi sebelumnya ada, gunakan header sebelumnya
                elif previous_headers is not None:
                    df.columns = previous_headers

                # Jika ada pending row dari halaman sebelumnya, gabungkan dengan baris pertama
                if pending_row is not None:
                    df.iloc[0] = df.iloc[0].combine_first(pending_row)
                    pending_row = None

                # Simpan baris terakhir jika tidak lengkap (mungkin terpotong di halaman berikutnya)
                last_row = df.iloc[-1]
                if last_row.isnull().sum() > 0:
                    pending_row = last_row
                    df = df[:-1]  # Hapus baris yang belum lengkap

                all_extracted_data.append(df)

    # Gabungkan semua tabel yang telah diperbaiki
    if all_extracted_data:
        final_df = pd.concat(all_extracted_data, ignore_index=True)
        final_df.to_excel(output_excel, index=False, header=False)
        print(f"✅ Semua tabel berhasil diekspor ke {output_excel}")
    else:
        print("⚠ Tidak ada tabel yang cocok ditemukan.")

# Contoh penggunaan
pdf_path = "014-2025-FK-01759_140225_KELAPA GADING_0065_K9_CAC.pdf"
output_excel = "output_tabel.xlsx"
extract_tables_from_pdf(pdf_path, output_excel)

File yang telah diperbaiki disimpan di: Fixed_Book2.xlsx


In [ ]:
import pandas as pd

# Load file Excel
file_path = "output_tabel.xlsx"
df = pd.read_excel(file_path, header=None)

# Temukan baris header (baris ganjil) dan baris nilai (baris genap)
for row in range(0, len(df) - 1, 2):
    header_row = df.iloc[row]  # Baris header
    value_row = df.iloc[row + 1]  # Baris nilai
    
    # Cek jika nilai tidak sejajar dengan header
    if value_row.first_valid_index() < header_row.first_valid_index():
        # Pindahkan nilai ke kolom yang sesuai
        corrected_values = value_row.dropna().values
        
        # Pastikan panjang nilai sesuai dengan jumlah header yang valid
        valid_headers = header_row.dropna().index
        for i, col in enumerate(valid_headers):
            if i < len(corrected_values):
                df.iat[row + 1, col] = corrected_values[i]  # Pindahkan nilai ke kolom yang benar
                
        # Kosongkan kolom lama yang tidak sesuai
        df.iloc[row + 1, :valid_headers[0]] = None

# Simpan hasil perbaikan
df.to_excel(file_path, index=False, header=False)
print(f"File telah disimpan di {file_path}")

Sheet2 berhasil dibuat dengan struktur yang diminta!


In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Baca file Excel
file_path = 'output_tabel.xlsx'
sheet1 = pd.read_excel(file_path, sheet_name='Sheet1', header=None)

# Inisialisasi list untuk menyimpan data
data_rows = []
current_row = {}
cols_needed = [
    "tanggal temuan", "cara temuan", "waktu pendeteksian", "nama kantor", "provinsi", "kota",
    "jenis kontributor", "kantor kontributor", "nama kontributor", "dokumen pendukung", 
    "no. identitas", "keterangan", "provinsi", "kota", "kecamatan", "pecahan", "tahun emisi",
    "no. seri 1", "no. seri 2", "jumlah lembar", "jumlah lembar terima", "subtotal",
    "jumlah dianalisa", "hasil analisa", "subtotal"
]

# Variabel tambahan untuk memastikan setiap tabel memiliki batasan yang jelas
found_first_subtotal = False

# Proses parsing data
for idx, row in sheet1.iterrows():
    for col_num, cell_value in enumerate(row):
        if cell_value in cols_needed:
            # Ambil nilai di bawah header jika ada
            if idx+1 < len(sheet1):
                next_val = sheet1.iloc[idx+1, col_num]
                
                # Penanganan khusus untuk "subtotal"
                if cell_value == 'subtotal':
                    if found_first_subtotal:
                        current_row['subtotal 2'] = next_val
                        data_rows.append(current_row)  # Simpan data sebagai satu tabel
                        current_row = {}  # Reset untuk tabel berikutnya
                        found_first_subtotal = False  # Reset flag subtotal pertama
                    else:
                        current_row['subtotal'] = next_val
                        found_first_subtotal = True
                elif cell_value == 'provinsi' and 'provinsi' in current_row:
                    current_row['provinsi_kontributor'] = next_val
                elif cell_value == 'kota' and 'kota' in current_row:
                    current_row['kota_kontributor'] = next_val
                else:
                    current_row[cell_value] = next_val

# Pastikan tabel terakhir tersimpan jika tidak ada subtotal kedua
if current_row:
    data_rows.append(current_row)

# Buat DataFrame
df = pd.DataFrame(data_rows)

# Cek apakah ada tabel tanpa "tanggal temuan", lalu pindahkan provinsi & kota ke provinsi_kontributor & kota_kontributor
if 'tanggal temuan' in df.columns:
    df['tanggal temuan'] = pd.to_datetime(df['tanggal temuan'], errors='coerce')
    mask_no_date = df['tanggal temuan'].isna()
    df.loc[mask_no_date, 'provinsi_kontributor'] = df.loc[mask_no_date, 'provinsi']
    df.loc[mask_no_date, 'kota_kontributor'] = df.loc[mask_no_date, 'kota']
    df.loc[mask_no_date, ['provinsi', 'kota']] = None  # Kosongkan kolom provinsi & kota jika "tanggal temuan" tidak ada

# Format ulang "tanggal temuan"
date_columns = ["tanggal temuan"]
for col in date_columns:
    if col in df.columns:
        df[col] = df[col].dt.strftime('%d-%m-%Y')

df.fillna(method='ffill', inplace=True)

# Simpan ke sheet baru
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, sheet_name='Sheet2', index=False)

print("Sheet2 berhasil dibuat dengan struktur yang diminta!")

Sheet2 berhasil dibuat dengan struktur yang diminta!


C:\Users\lenovo\AppData\Local\Temp\ipykernel_9588\1854577445.py:68: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
